In [1]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

In [2]:
# Read the data
means_df = pd.read_csv('/Users/mahinbindra/Downloads/Means.csv')
covariances_df = pd.read_csv('/Users/mahinbindra/Downloads/Covariances.csv')

In [3]:
# Set the index to the first column which likely represents asset names
covariances_df.set_index(covariances_df.columns[0], inplace=True)

In [4]:
# Initialize model
m = gp.Model("portfolio_optimization")

Set parameter Username
Academic license - for non-commercial use only - expires 2025-01-15


In [5]:
# Number of assets
n_assets = len(means_df)

In [6]:
# Decision variables: fraction of total wealth invested in each asset
x = m.addVars(n_assets, lb=0, ub=1, name="x")

In [7]:
# Objective function: Minimize portfolio risk
portfolio_risk = gp.quicksum(covariances_df.iat[i, j] * x[i] * x[j] 
                             for i in range(n_assets) for j in range(n_assets))
m.setObjective(portfolio_risk, GRB.MINIMIZE)

In [8]:
# Constraint 1: Expected return >= 8%
m.addConstr(gp.quicksum(x[i] * means_df.iloc[i]['Returns'] for i in range(n_assets)) >= 0.08, "min_return")

# Constraint 2: Balance between specific assets
m.addConstr(gp.quicksum(x[i] for i in range(10)) - gp.quicksum(x[i] for i in range(90, 100)) <= 0.005, "balance_upper")
m.addConstr(gp.quicksum(x[i] for i in range(90, 100)) - gp.quicksum(x[i] for i in range(10)) <= 0.005, "balance_lower")

# Constraint 3: Investment ratios between asset groups
m.addConstr(gp.quicksum(x[i] for i in range(10, 30)) >= 2 * gp.quicksum(x[i] for i in range(70, 90)), "ratio_investment")

# Constraint 4: Maximum investment in a group
m.addConstr(gp.quicksum(x[i] for i in range(40, 60)) <= 0.65, "max_group_investment")

# Constraint 5: Minimum investment in a group
m.addConstr(gp.quicksum(x[i] for i in range(60, 70)) >= 0.10, "min_group_investment")

# Constraint: Sum of investments = 1
m.addConstr(gp.quicksum(x[i] for i in range(n_assets)) == 1, "total_investment")

<gurobi.Constr *Awaiting Model Update*>

In [9]:
# Optimize model
m.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.0.0 23A344)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 7 rows, 100 columns and 310 nonzeros
Model fingerprint: 0xa8fcfafc
Model has 5050 quadratic objective terms
Coefficient statistics:
  Matrix range     [3e-04, 2e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e-06, 3e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-03, 1e+00]
Presolve removed 1 rows and 0 columns
Presolve time: 0.02s
Presolved: 6 rows, 101 columns, 291 nonzeros
Presolved model has 5050 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 63
 AA' NZ     : 2.316e+03
 Factor NZ  : 2.415e+03
 Factor Ops : 1.119e+05 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   6.00524157e-14 -3.9062

In [10]:
# Display results
if m.status == GRB.OPTIMAL:
    portfolio = m.getAttr('x', x)
    for asset, fraction in portfolio.items():
        print(f"Asset {asset}: {fraction:.2%}")

Asset 0: 0.00%
Asset 1: 0.00%
Asset 2: 0.00%
Asset 3: 0.00%
Asset 4: 0.00%
Asset 5: 0.00%
Asset 6: 0.00%
Asset 7: 0.00%
Asset 8: 0.00%
Asset 9: 0.00%
Asset 10: 0.00%
Asset 11: 0.00%
Asset 12: 5.25%
Asset 13: 0.00%
Asset 14: 0.00%
Asset 15: 0.00%
Asset 16: 0.00%
Asset 17: 0.00%
Asset 18: 0.00%
Asset 19: 0.00%
Asset 20: 0.00%
Asset 21: 0.00%
Asset 22: 0.00%
Asset 23: 0.00%
Asset 24: 0.00%
Asset 25: 0.00%
Asset 26: 0.00%
Asset 27: 0.00%
Asset 28: 0.00%
Asset 29: 0.00%
Asset 30: 9.99%
Asset 31: 0.00%
Asset 32: 6.16%
Asset 33: 0.00%
Asset 34: 0.00%
Asset 35: 0.00%
Asset 36: 1.08%
Asset 37: 0.00%
Asset 38: 0.00%
Asset 39: 0.00%
Asset 40: 0.00%
Asset 41: 0.00%
Asset 42: 16.29%
Asset 43: 40.39%
Asset 44: 0.00%
Asset 45: 0.00%
Asset 46: 0.00%
Asset 47: 0.00%
Asset 48: 0.00%
Asset 49: 0.00%
Asset 50: 0.00%
Asset 51: 0.00%
Asset 52: 6.74%
Asset 53: 0.00%
Asset 54: 0.00%
Asset 55: 0.00%
Asset 56: 0.00%
Asset 57: 1.47%
Asset 58: 0.00%
Asset 59: 0.00%
Asset 60: 4.66%
Asset 61: 3.30%
Asset 62: 0.00%


In [11]:
# Display results
if m.status == GRB.OPTIMAL:
    portfolio = m.getAttr('x', x)
    # Calculate and print the optimal portfolio return
    optimal_return = sum(portfolio[i] * means_df.iloc[i]['Returns'] for i in range(n_assets))
    print(f"Optimal Portfolio Return: {optimal_return:.2%}")
    for asset, fraction in portfolio.items():
        print(f"Asset {asset}: {fraction:.2%}")

Optimal Portfolio Return: 8.00%
Asset 0: 0.00%
Asset 1: 0.00%
Asset 2: 0.00%
Asset 3: 0.00%
Asset 4: 0.00%
Asset 5: 0.00%
Asset 6: 0.00%
Asset 7: 0.00%
Asset 8: 0.00%
Asset 9: 0.00%
Asset 10: 0.00%
Asset 11: 0.00%
Asset 12: 5.25%
Asset 13: 0.00%
Asset 14: 0.00%
Asset 15: 0.00%
Asset 16: 0.00%
Asset 17: 0.00%
Asset 18: 0.00%
Asset 19: 0.00%
Asset 20: 0.00%
Asset 21: 0.00%
Asset 22: 0.00%
Asset 23: 0.00%
Asset 24: 0.00%
Asset 25: 0.00%
Asset 26: 0.00%
Asset 27: 0.00%
Asset 28: 0.00%
Asset 29: 0.00%
Asset 30: 9.99%
Asset 31: 0.00%
Asset 32: 6.16%
Asset 33: 0.00%
Asset 34: 0.00%
Asset 35: 0.00%
Asset 36: 1.08%
Asset 37: 0.00%
Asset 38: 0.00%
Asset 39: 0.00%
Asset 40: 0.00%
Asset 41: 0.00%
Asset 42: 16.29%
Asset 43: 40.39%
Asset 44: 0.00%
Asset 45: 0.00%
Asset 46: 0.00%
Asset 47: 0.00%
Asset 48: 0.00%
Asset 49: 0.00%
Asset 50: 0.00%
Asset 51: 0.00%
Asset 52: 6.74%
Asset 53: 0.00%
Asset 54: 0.00%
Asset 55: 0.00%
Asset 56: 0.00%
Asset 57: 1.47%
Asset 58: 0.00%
Asset 59: 0.00%
Asset 60: 4.66%
